## Pytorch Lightning을 활용한 CNN MNIST

In [2]:
import torch
import torch.nn as nn
import lightning.pytorch as pl
import torchmetrics

from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms, datasets

In [3]:
## 코드작성

class LMNISTModel(pl.LightningModule):

    # 모델이 활용하는 레이어들 선언
    def __init__(self):
        super().__init__()

        # Metric
        self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)

        # Feature Extractor

        # MNIST 이미지의 크기: (1, 28, 28) -> (채널, width, height)
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.maxpool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, 3)
        self.maxpool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 64, 3)

        # Classification
        self.linear1 = nn.Linear(64 * 3 * 3, 64)
        self.linear2 = nn.Linear(64, 10)

    # 데이터를 모델에 흘려보낼 때 어떤 연산들을 통해서 결과를 내보낼지.
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.maxpool1(x)

        x = torch.relu(self.conv2(x))
        x = self.maxpool2(x)

        x = torch.relu(self.conv3(x))

        # Flatten
        x = nn.Flatten()(x)

        # classification
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)

        # softmax
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy(self, logits, lables):
        return nn.functional.nll_loss(logits, lables)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        self.log('train_loss', loss)
        self.train_acc(logits, y)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False)

        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        self.log('val_loss', loss)
        self.val_acc(logits, y)
        self.log('val_acc', self.val_acc, on_step=True, on_epoch=True)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        self.log('test_loss', loss)
        self.test_acc(logits, y)
        self.log('test_acc', self.test_acc, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

## 데이터 가공

In [4]:
# 데이터 정규화
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 데이터 준비
mnist_train = MNIST(root="Dataset", train=True, download=True, transform=transform)
mnist_train, mnist_val = random_split(mnist_train, [0.8, 0.2])
mnist_test = MNIST(root="Dataset", train=False, download=True, transform=transform)

# 데이터 로더
train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=128)
test_loader = DataLoader(mnist_test, batch_size=128)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.14MB/s]


Extracting CNN/Dataset\MNIST\raw\train-images-idx3-ubyte.gz to CNN/Dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 146kB/s]


Extracting CNN/Dataset\MNIST\raw\train-labels-idx1-ubyte.gz to CNN/Dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.40MB/s]


Extracting CNN/Dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to CNN/Dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.56MB/s]

Extracting CNN/Dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to CNN/Dataset\MNIST\raw



In [5]:
model = LMNISTModel()
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=50)

trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0      | train
1 | val_acc   | MulticlassAccuracy | 0      | train
2 | test_acc  | MulticlassAccuracy | 0      | train
3 | conv1     | Conv2d             | 320    | train
4 | maxpool1  | MaxPool2d          | 0      | train
5 | conv2     | Conv2d             | 18.5 K | train
6 | maxpool2  | MaxPool2d          | 0   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [7]:
trainer.test(model, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9904999732971191     │
│         test_loss         │    0.06737816333770752    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.06737816333770752, 'test_acc_epoch': 0.9904999732971191}]